# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [39]:
import pandas as pd

# load data data
df = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv')

df.head()

# Delete the unnamed index columns
df = df.drop(columns=['Unnamed: 0'], axis=1)

#Standardize the column names
df.columns = df.columns.str.lower().str.replace(" ", "_")



In [41]:
# checking info about marketing_customer_df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10910 entries, 0 to 10909
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   customer                       10910 non-null  object 
 1   state                          10279 non-null  object 
 2   customer_lifetime_value        10910 non-null  float64
 3   response                       10279 non-null  object 
 4   coverage                       10910 non-null  object 
 5   education                      10910 non-null  object 
 6   effective_to_date              10910 non-null  object 
 7   employmentstatus               10910 non-null  object 
 8   gender                         10910 non-null  object 
 9   income                         10910 non-null  int64  
 10  location_code                  10910 non-null  object 
 11  marital_status                 10910 non-null  object 
 12  monthly_premium_auto           10910 non-null 

In [43]:
print(df.columns)

Index(['customer', 'state', 'customer_lifetime_value', 'response', 'coverage',
       'education', 'effective_to_date', 'employmentstatus', 'gender',
       'income', 'location_code', 'marital_status', 'monthly_premium_auto',
       'months_since_last_claim', 'months_since_policy_inception',
       'number_of_open_complaints', 'number_of_policies', 'policy_type',
       'policy', 'renew_offer_type', 'sales_channel', 'total_claim_amount',
       'vehicle_class', 'vehicle_size', 'vehicle_type'],
      dtype='object')


In [45]:
# 1. Create a new DataFrame that only includes customers who have a total_claim_amount >$1,000 and response == "Yes".
df_filtered = df[df['total_claim_amount'] > 1000]
df_filtered = df_filtered[df_filtered['response'] == 'Yes']

df_filtered

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
189,OK31456,California,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,51643,...,0.0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,NaN
236,YJ16163,Oregon,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,51643,...,0.0,1,Special Auto,Special L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,A
419,GW43195,Oregon,25807.063000,Yes,Extended,College,2/13/11,Employed,F,71210,...,1.0,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,Luxury Car,Small,A
442,IP94270,Arizona,13736.132500,Yes,Premium,Master,2/13/11,Disabled,F,16181,...,0.0,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,SUV,Medsize,A
587,FJ28407,California,5619.689084,Yes,Premium,High School or Below,1/26/11,Unemployed,M,0,...,0.0,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,SUV,Medsize,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10351,FN44127,Oregon,3508.569533,Yes,Extended,College,1/5/11,Medical Leave,M,20978,...,1.0,1,Personal Auto,Personal L2,Offer2,Branch,1176.278800,Four-Door Car,Small,NaN
10373,XZ64172,Oregon,10963.957230,Yes,Premium,High School or Below,2/8/11,Employed,M,55687,...,0.0,1,Corporate Auto,Corporate L2,Offer1,Agent,1324.800000,Luxury SUV,Medsize,NaN
10487,IX60941,Oregon,3508.569533,Yes,Extended,College,1/5/11,Medical Leave,M,20978,...,1.0,1,Personal Auto,Personal L3,Offer2,Branch,1176.278800,Four-Door Car,Small,NaN
10565,QO62792,Oregon,7840.165778,Yes,Extended,College,1/14/11,Employed,M,58414,...,2.0,1,Personal Auto,Personal L3,Offer2,Agent,1008.000000,NaN,NaN,NaN


In [52]:
#2 Using the original Dataframe,  the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. 

customers_yes=df[df['response']=='Yes']

# Calculate the average claim amount segmented by policy type and gender to 2d.p.
average_claim = customers_yes.groupby(['policy_type', 'gender'])['total_claim_amount'].mean().round(2)
average_claim


policy_type     gender
Corporate Auto  F         433.74
                M         408.58
Personal Auto   F         452.97
                M         457.01
Special Auto    F         453.28
                M         429.53
Name: total_claim_amount, dtype: float64

In [ ]:
#for corporate and special autos, females have on average higher claim amount than males. For personal autos, the average is roughly the same.

In [58]:
#3  Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.
customers_count = df['state'].value_counts()

# Filter for states with more than 500 customers
filtered_customers_count = customers_count[customers_count > 500]
print(filtered_customers_count)

state
California    3552
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
Name: count, dtype: int64


In [61]:
#4. Find the maximum, minimum, and median customer lifetime value by education level and gender.
grouped_stats = df.groupby(['education', 'gender'])['customer_lifetime_value'].describe().round(2)
grouped_stats



count     mean      std      min      25%  \
education            gender                                               
Bachelor             F       1700.0  7874.27  6614.11  1904.00  4118.59   
                     M       1572.0  7703.60  6658.46  1898.01  3842.35   
College              F       1608.0  7748.82  6678.30  1898.68  3866.80   
                     M       1596.0  8052.46  6716.96  1918.12  4035.90   
Doctor               F        200.0  7328.51  6930.56  2395.57  3531.28   
                     M        201.0  7415.33  5460.94  2267.60  3557.37   
High School or Below F       1590.0  8675.22  7497.85  2144.92  4158.28   
                     M       1556.0  8149.69  7190.62  1940.98  4197.68   
Master               F        475.0  8157.05  6621.77  2417.78  4061.87   
                     M        412.0  8168.83  7362.16  2272.31  3933.03   

                                 50%      75%       max  
education            gender                              
Bachelor             F       5640.51  8723.24  73225.96  
                     M       5548.03  8646.70  67907.27  
College              F       5623.61  8714.92  61850.19  
                     M       6005.85  9157.76  61134.68  
Doctor               F       5332.46  7662.70  44856.11  
                     M       5577.67  8828.38  32677.34  
High School or Below F       6039.55  9457.34  55277.45  
                     M       6286.73  9002.19  83325.38  
Master               F       5729.86  9264.04  51016.07  
                     M       5579.10  9031.08  50568.26

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [64]:
df['effective_to_date'] = pd.to_datetime(df['effective_to_date'], format='%Y-%m-%d')
print(df['effective_to_date'].head())

0   2011-02-18
1   2011-01-18
2   2011-02-10
3   2011-01-11
4   2011-01-17
Name: effective_to_date, dtype: datetime64[ns]


In [67]:
policies_by_state_month = df.pivot_table(index='state', columns='month', aggfunc='size', fill_value=0)
# Group by both state and month, then count the rows in each group
policies_grouped = df.groupby(['state', 'month']).size()


In [69]:
# Unstack the month level to create a table with states as rows and months as columns
policies_by_state_month = policies_grouped.unstack(level='month', fill_value=0)

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here